In [1]:
from datasets import load_dataset, load_metric

In [2]:
import numpy as np

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [4]:
train_dataset = load_dataset('bakhitovd/data_science_arxiv', split='train')

Found cached dataset json (C:/Users/bakhi/.cache/huggingface/datasets/bakhitovd___json/bakhitovd--data_science_arxiv-d562cf23e63fbcaf/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


In [5]:
test_dataset = load_dataset('bakhitovd/data_science_arxiv', split='test')

Found cached dataset json (C:/Users/bakhi/.cache/huggingface/datasets/bakhitovd___json/bakhitovd--data_science_arxiv-d562cf23e63fbcaf/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


In [6]:
val_dataset = load_dataset('bakhitovd/data_science_arxiv', split='validation')

Found cached dataset json (C:/Users/bakhi/.cache/huggingface/datasets/bakhitovd___json/bakhitovd--data_science_arxiv-d562cf23e63fbcaf/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


In [6]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

In [14]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

In [22]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [7]:
max_input_length = 7168 # it is calculated
max_output_length = 512
batch_size = 1

In [8]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["article"],
        padding='max_length',
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [18]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [9]:
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract"],
)

train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract"],
)

val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

Loading cached processed dataset at C:\Users\bakhi\.cache\huggingface\datasets\bakhitovd___json\bakhitovd--data_science_arxiv-d562cf23e63fbcaf\0.0.0\fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e\cache-f77776da8f89a889.arrow


In [17]:
rouge = load_metric("rouge")

In [19]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./",
    logging_steps=250,
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
)

In [20]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

Using cuda_amp half precision backend


In [20]:
trainer.train()

C:\Users\bakhi\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 30280
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 22710
  Number of trainable parameters = 161844480
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
500,2.862800,2.743932,0.182400,0.128400,0.145200
1000,2.799700,2.661958,0.189200,0.131100,0.149000
1500,2.730300,2.655223,0.192400,0.125600,0.146200
2000,2.641300,2.617599,0.185700,0.133100,0.149600
2500,2.616900,2.581202,0.186900,0.140400,0.154000
3000,2.610500,2.582049,0.183200,0.147500,0.156700
3500,2.627900,2.552020,0.182000,0.132500,0.147400
4000,2.558300,2.513738,0.175000,0.152900,0.155800
4500,2.587900,2.521286,0.180100,0.147900,0.155700
5000,2.514300,2.480737,0.185600,0.152800,0.160900


C:\Users\bakhi\anaconda3\lib\site-packages\transformers\trainer.py:1820: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 1196
  Batch size = 1
Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500\config.json
Model weights saved in ./checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./checkpoint-500\tokenizer_config.json
Special tokens file saved in ./checkpoint-500\special_tokens_map.json
C:\Users\bakhi\anaconda3\lib\site-packages\transformers\trainer.py:1820: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future r

TrainOutput(global_step=22710, training_loss=2.3153236849409846, metrics={'train_runtime': 495743.7726, 'train_samples_per_second': 0.183, 'train_steps_per_second': 0.046, 'total_flos': 4.292515625906995e+17, 'train_loss': 2.3153236849409846, 'epoch': 3.0})

In [20]:
led.save_pretrained("LED_7k_epoch_3")

Configuration saved in LED_7k_epoch_3\config.json
Model weights saved in LED_7k_epoch_3\pytorch_model.bin
